In [22]:
import os

old_folder = r'D:\Shiwei\20250216-hM1_CTP14_from_Cx28_sample5\Alignment'

before_position_file = os.path.join(old_folder, '10x_positions_before.txt')

new_folder = r'D:\Shiwei\20250216-hM1_CTP14_from_Cx28_sample5\Alignment'

after_position_file = os.path.join(new_folder, '10x_positions_after.txt')

os.path.isfile(before_position_file), os.path.isfile(after_position_file)



(True, True)

# Calculate rotation matrix

In [23]:
import numpy as np
import os
# 1. alignment for manually picked points
def align_manual_points(pos_file_before, pos_file_after,
                        save=True, save_folder=None, save_filename='', verbose=True):
    """Function to align two manually picked position files, 
    they should follow exactly the same order and of same length.
    Inputs:
        pos_file_before: full filename for positions file before translation
        pos_file_after: full filename for positions file after translation
        save: whether save rotation and translation info, bool (default: True)
        save_folder: where to save rotation and translation info, None or string (default: same folder as pos_file_before)
        save_filename: filename specified to save rotation and translation points
        verbose: say something! bool (default: True)
    Outputs:
        R: rotation for positions, 2x2 array
        T: traslation of positions, array of 2
    Here's example for how to translate points
        translated_ps_before = np.dot(ps_before, R) + t
    """
    # load position_before
    if os.path.isfile(pos_file_before):
        ps_before = np.loadtxt(pos_file_before, delimiter=',')

    # load position_after
    if os.path.isfile(pos_file_after):
        ps_after = np.loadtxt(pos_file_after, delimiter=',')

    # do SVD decomposition to get best fit for rigid-translation
    c_before = np.mean(ps_before, axis=0)
    c_after = np.mean(ps_after, axis=0)
    H = np.dot((ps_before - c_before).T, (ps_after - c_after))
    U, _, V = np.linalg.svd(H)  # do SVD
    # calcluate rotation
    R = np.dot(V, U.T).T
    if np.linalg.det(R) < 0:
        R[:, -1] = -1 * R[:, -1]
    # calculate translation
    t = - np.dot(c_before, R) + c_after
    # here's example for how to translate points
    # translated_ps_before = np.dot(ps_before, R) + t
    # save
    if save:
        if save_folder is None:
            save_folder = os.path.dirname(pos_file_before)
        if not os.path.exists(save_folder):
            os.makedirs(save_folder)
        if len(save_filename) > 0:
            save_filename += '_'
        rotation_name = os.path.join(save_folder, save_filename+'rotation')
        translation_name = os.path.join(
            save_folder, save_filename+'translation')
        np.save(rotation_name, R)
        np.save(translation_name, t)

    return R, t

In [24]:
R, T = align_manual_points(before_position_file, after_position_file, save=False, save_folder=new_folder)
R, T

(array([[ 0.99999302, -0.00373584],
        [ 0.00373584,  0.99999302]]),
 array([-2411.09167153,   396.86670492]))

In [25]:
np.arccos(R[0,1])

1.5745321746245373

# Translate 60x positions

In [26]:
old_positions = np.loadtxt(os.path.join(os.path.dirname(old_folder), 'positions_all.txt'), delimiter=',')

In [27]:
new_positions = np.dot(old_positions, R) + T
print(new_positions)

[[-5868.71542144  1869.19410536]
 [-5869.48126846  1664.1955359 ]
 [-5870.24711548  1459.19696645]
 ...
 [  685.87787194   409.69701442]
 [-1399.74974252  1728.52778685]
 [-6123.5544053   1990.39698859]]


In [28]:
save_filename = os.path.join(new_folder, 'translated_positions_all.txt')
print(save_filename)
np.savetxt(save_filename, new_positions, fmt='%.2f', delimiter=',')

D:\Shiwei\20250216-hM1_CTP14_from_Cx28_sample5\Alignment\translated_positions_all.txt


# Manual adjust positions

In [29]:
manual_folder = r'D:\Shiwei\20250216-hM1_CTP14_from_Cx28_sample5\Alignment'
translated_manual_positions_file = os.path.join(manual_folder,'60x_positions_before.txt')
translated_manual_positions = np.loadtxt(translated_manual_positions_file, delimiter=',')
print(translated_manual_positions)

[[-5868.72  1869.19]
 [-1982.93  -605.34]
 [-1399.75  1728.53]]


In [30]:
#manual_folder = r'D:\Shiwei\20240703-hM1_CTP14_from_Cx28_sample3\Alignment'
real_manual_positions_file = os.path.join(manual_folder,'60x_positions_after.txt')
real_manual_positions = np.loadtxt(real_manual_positions_file, delimiter=',')
print(real_manual_positions)


[[-5887.9   1873.7 ]
 [-1997.85  -590.05]
 [-1419.8   1744.5 ]]


In [31]:

manual_shifts = np.array(real_manual_positions) - translated_manual_positions[:len(real_manual_positions)]
manual_shifts

array([[-19.18,   4.51],
       [-14.92,  15.29],
       [-20.05,  15.97]])

In [32]:
manual_shift = np.mean(manual_shifts, axis=0)
manual_shift

array([-18.05      ,  11.92333333])

In [33]:
adjusted_new_positions = new_positions + manual_shift

In [34]:
adjusted_new_positions[0], old_positions[0]

(array([-5886.76542144,  1881.11743869]), array([-3463.1,  1459.4]))

In [35]:
adj_save_filename = os.path.join(new_folder, 'adjusted_translated_positions_all.txt')
print(adj_save_filename)
np.savetxt(adj_save_filename, adjusted_new_positions, fmt='%.2f', delimiter=',')

D:\Shiwei\20250216-hM1_CTP14_from_Cx28_sample5\Alignment\adjusted_translated_positions_all.txt


In [36]:
translated_manual_positions = np.dot(manual_positions, R) + T
print('translated manual positions:')
print(np.round(translated_manual_positions, 1))

translated manual positions:
[[ 474.9 1293.1]
 [6011.4  691.3]
 [2917.  2511.9]]
